# Tutorial 1: Instalación e introducción al ML en química

### Instalación de la librería DeepChem

In [2]:
pip install --upgrade typing-extensions

  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.21.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatible.


In [4]:
!pip install --pre deepchem[tensorflow]

  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.8.0
    Uninstalling typing-extensions-4.8.0:
      Successfully uninstalled typing-extensions-4.8.0


In [5]:
import deepchem as dc
dc.__version__

NameError: name '_C' is not defined

**Pasos básicos para el entrenamiento de un modelo**
- Selección del dataset de entrenamiento de nuestro modelo 
- Creación del modelo 
- Entrenamiento del modelo
- Evaluación del modelo en un conjunto de datos específicos para las pruebas
- Hacer predicciones sobre nuevos datos

## Paso 1. Selección del dataset de entrenamiento

El primer problema al que nos enfrentaremos y buscaremos solucionar con DeepChem es el de **predecir la solubilidad de pequeñas moléculas dada su fórmula química.** Esto es una propiedad importante en el desarrollo de fármacos, ya que si el fármaco propuesto **no es lo suficientemente soluble, probablemente no llegue hasta el torrente sanguineo para tener un efecto terapeutico.**

Lo primero que necesitamos es un **dataset de medidas de solubilidades para moléculas reales.** Uno de los componentes principales de DeepChem es **MoleculeNet**, una **colección de quimica y molecular datasets.** Para ello, usaremos la **Delaney solubility dataset.** La propiedad de solubilidad en este dataset esta establecida como **log(solubility)** donde la solubilidad esta medida en **moles/litro**

In [10]:
#De la librería DeepChem cargamos el dataset delaney. En el argumento 
#Featurizer indicaremos la representación que queremos usar para las
#moléculas. En este caso, hemos indicado Grafos. La manera en la que es
#representada los datos es conocida como Featurize.
tasks, datasets, transformers = dc.molnet.load_delaney(featurizer='GraphConv')
train_dataset, valid_dataset, test_dataset = datasets
#Del dataset que hemos cargado vamos a separarlo en un dataset de
#entrenamiento, otro de validación y otro de testeo

Podemos observar que del dataset que hemos cargado lo separamos en tres conjuntos, uno de entrenamiento, otro de validación y otro de pruebas.
* **Conjunto de Entrenamiento (train_dataset):**
  * Este conjunto se utiliza para entrenar el modelo. Contiene ejemplos de datos etiquetados (entradas y las respuestas correctas).
  * El modelo aprende de estos datos y ajusta sus parámetros durante el proceso de entrenamiento para hacer predicciones precisas.
  * Por lo general, el conjunto de entrenamiento es el más grande, ya que se necesita una cantidad significativa de datos para entrenar un modelo de manera efectiva.
* **Conjunto de Entrenamiento (train_dataset):**
  * Este conjunto se utiliza para ajustar hiperparámetros del modelo y evaluar su rendimiento durante el entrenamiento.
  * Se utiliza para comprobar si el modelo está sobreajustando (overfitting) los datos de entrenamiento.
  * Los hiperparámetros (como la tasa de aprendizaje) se ajustan utilizando este conjunto para optimizar el rendimiento del modelo.

* **Conjunto de Prueba (test_dataset):**
    * Este conjunto se utiliza para evaluar el rendimiento final del modelo después de que se ha entrenado y ajustado con el conjunto de entrenamiento y el conjunto de validación.
    * Los datos de prueba son completamente independientes de los datos de entrenamiento y validación, lo que permite evaluar la capacidad del modelo para generalizar a nuevos datos.

## Paso 2. Creación del modelo

Una vez que tenemos los datos listos procedemos a crear el modelo. Usaremos un modelo llamado "graph convolutional network" o "graphcov". Este es un modelo para procesar datos estructurados o gráficos, como moléculas o proteínas

In [13]:
model = dc.models.GraphConvModel(n_tasks=1, mode='regression', dropout=0.2)

Los distintos valores de entrada al parámetro significan lo siguiente:

* **dc.models.GraphConvModel:** Esto crea un objeto de modelo de tipo GraphConvModel de DeepChem. GraphConvModel es un modelo diseñado para procesar datos estructurados o gráficos, como moléculas en química o proteínas en biología.

* **n_tasks=1:** Este parámetro especifica el número de tareas o salidas que se esperan del modelo. En este caso, se configura en 1, lo que sugiere que el modelo se está entrenando para una tarea específica. Si se tratara de una tarea de clasificación con varias clases, este número sería mayor.

* **mode='regression':** El modo de operación del modelo se establece en "regression", lo que indica que se está configurando el modelo para realizar una tarea de regresión. En una tarea de regresión, el modelo predice valores numéricos continuos en lugar de etiquetas categóricas.

* **dropout=0.2:** El valor 0.2 se refiere a la tasa de dropout, que es una técnica de regularización utilizada en redes neuronales. Un valor de 0.2 significa que durante el entrenamiento, el 20% de las conexiones en la red se desactivarán aleatoriamente en cada paso de entrenamiento para prevenir el sobreajuste.

## Paso 3. Entrenamiento del modelo

Una vez generado el modelo el siguiente paso es entrenarlo. Para ello, haremos uso de fit.

In [14]:
model.fit(train_dataset, nb_epoch=100) #Entrenamos el modelo usando el
                                       #conjunto de entrenamiento

0.11181475639343262

El argumento **nb_epoch=100** indica que el modelo se entrenará durante 100 épocas. Durante cada una de estas 100 épocas, el modelo procesará todo el conjunto de datos de entrenamiento train_dataset para aprender a realizar predicciones más precisas.

El uso de múltiples épocas es común en el entrenamiento de modelos de aprendizaje automático para asegurar que el modelo haya tenido la oportunidad de aprender de manera efectiva a partir de los datos. Sin embargo, la cantidad óptima de épocas puede variar según el problema y el modelo, por lo que a menudo se realiza ajuste fino de este valor para lograr un equilibrio entre un entrenamiento efectivo y la prevención del sobreajuste.

## Paso 4. Evaluación del modelo

El siguiente paso tras entrenar el modelo es evaluarlo. Para ello, evaluaremos el modelo tanto para el conjunto de entrenamiento como para el conjunto de pruebas. Para ello, estableceremos la métrica de la correlación de Pearson, es decir, r^2. 

In [15]:
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
print("Training set score:", model.evaluate(train_dataset, [metric], transformers))
print("Test set score:", model.evaluate(test_dataset, [metric], transformers))

Training set score: {'pearson_r2_score': 0.9176339136802922}
Test set score: {'pearson_r2_score': 0.6172329850299577}


Vemos que tenemos muy buen rendimiento en el conjunto de entrenamiento pero algo malo en el conjunto de pruebas. Esto es lo que se le conoce como sobreajuste u overfitting.

Aún así, nuestra métrica no esta mal. Un modelo que produjese predicciones aleatorias tendria una correlación de 0, mientras que una que hiciese predicciones perfectas tendria correlación de 1. Nuestro modelo no lo hace tan mal

## Paso 5. Predicciones con nuevas moléculas

Haremos predicciones de las primeras diez moléculas del conjunto de pruebas. Para cada una de ellas, imprimiremos su estructura química (SMILES string) y la solubilidad que ha predecido (log(solubility)). Tambien mostraremos la solubilidad real dentro del conjunto de entrenamiento

In [21]:
#Realizamos las predicciones con nuestro modelo.
solubilities = model.predict_on_batch(test_dataset.X[:10])

#Recorremos las tres listas simultaneamente e imprimimos el SMILES de la
#molécula, la predicción de la solubilidad y la solubilidad real.
for molecule, solubility, test_solubility in zip(test_dataset.ids, solubilities, test_dataset.y):
    print(solubility, test_solubility, molecule)

[-1.645381] [-1.60114461] c1cc2ccc3cccc4ccc(c1)c2c34
[1.0510993] [0.20848251] Cc1cc(=O)[nH]c(=S)[nH]1
[-0.47844103] [-0.01602738] Oc1ccc(cc1)C2(OC(=O)c3ccccc23)c4ccc(O)cc4 
[-2.1627705] [-2.82191713] c1ccc2c(c1)cc3ccc4cccc5ccc2c3c45
[-1.4858996] [-0.52891635] C1=Cc2cccc3cccc1c23
[1.7102858] [1.10168349] CC1CO1
[-0.52068216] [-0.88987406] CCN2c1ccccc1N(C)C(=S)c3cccnc23 
[-0.7695658] [-0.52649706] CC12CCC3C(CCc4cc(O)ccc34)C2CCC1=O
[-1.2130748] [-0.76358725] Cn2cc(c1ccccc1)c(=O)c(c2)c3cccc(c3)C(F)(F)F
[0.84965193] [-0.64020358] ClC(Cl)(Cl)C(NC=O)N1C=CN(C=C1)C(NC=O)C(Cl)(Cl)Cl 
